Cyber Bullying - https://www.kaggle.com/code/kasanagottukumar/cyber-bullying/notebook

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/multilabel_classification'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
import numpy as np
import pandas as pd

import emoji
import re, string
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))


from nltk.stem import WordNetLemmatizer,PorterStemmer
from sklearn.model_selection import train_test_split
from nltk import word_tokenize
from tensorflow.keras.preprocessing.text import Tokenizer

from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from tensorflow.keras.models import Model

from collections import Counter
from gensim.models import Word2Vec
import matplotlib.pyplot as plt
import seaborn as sns

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score

2023-07-24 13:23:29.173292: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-24 13:23:31.205267: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [4]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /nas.dbms/buyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [5]:
df = pd.read_csv("cyberbullying_tweets.csv")
df

,tweet_text,cyberbullying_type
0,"In other words #katandandre, your food was cra...",not_cyberbullying
1,Why is #aussietv so white? #MKR #theblock #ImA...,not_cyberbullying
2,@XochitlSuckkks a classy whore? Or more red ve...,not_cyberbullying
3,"@Jason_Gio meh. :P thanks for the heads up, b...",not_cyberbullying
4,@RudhoeEnglish This is an ISIS account pretend...,not_cyberbullying
...,...,...
47687,"Black ppl aren't expected to do anything, depe...",ethnicity
47688,Turner did not withhold his disappointment. Tu...,ethnicity
47689,I swear to God. This dumb nigger bitch. I have...,ethnicity
47690,Yea fuck you RT @therealexel: IF YOURE A NIGGE...,ethnicity


In [6]:
df['cyberbullying_type'].unique()

array(['not_cyberbullying', 'gender', 'religion', 'other_cyberbullying',
       'age', 'ethnicity'], dtype=object)

Data cleaning

In [7]:
import re
def remove_special_character_And_punctuation(text):
    clean_txt = re.sub(r"[^\w\s]","",text)
    return clean_txt
df['tweet_text'] = df['tweet_text'].apply(remove_special_character_And_punctuation)

In [8]:
import nltk
def tokenization(text):
    return nltk.word_tokenize(text)

df['Tokens'] = df['tweet_text'].apply(tokenization)

In [9]:
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     /nas.dbms/buyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:
def remove_stopword(list_of_word):
    filtered_word = [word  for word in list_of_word if word.lower() not in stop_words]
    return filtered_word

df['Process_Tokens'] = df['Tokens'].apply(remove_stopword)

In [11]:
#these techniques help consolidate similar words and reduce vocabulary size
from nltk.stem import WordNetLemmatizer , PorterStemmer
from nltk.corpus import wordnet
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     /nas.dbms/buyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [13]:
def stemming(tokens):
    stemmer = PorterStemmer()
    stemmed_tokens = [stemmer.stem(token) for token in tokens]
    return stemmed_tokens
df['stemmed_Tokens'] = df['Process_Tokens'].apply(stemming)

In [14]:
#handling numerical data
def tokens_to_word(li):
    return ' '.join(li)
df['tt'] = df['stemmed_Tokens'].apply(tokens_to_word)

In [15]:
df['cyberbullying_type'].value_counts().index

Index(['religion', 'age', 'gender', 'ethnicity', 'not_cyberbullying',
       'other_cyberbullying'],
      dtype='object', name='cyberbullying_type')

In [16]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
df['cyberbullying_type'] = label_encoder.fit_transform(df['cyberbullying_type'])
df['cyberbullying_type']

0        3
1        3
2        3
3        3
4        3
        ..
47687    1
47688    1
47689    1
47690    1
47691    1
Name: cyberbullying_type, Length: 47692, dtype: int64

In [17]:
texts = df['tt']
labels = df['cyberbullying_type']

In [18]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
X_train, X_test, y_train, y_test = train_test_split(texts, labels, test_size=0.2, random_state=42)

Bag-of-Words (BoW): This technique represents text as a "bag" of individual words or tokens, disregarding the order of words. Each document is encoded as a vector, where each dimension corresponds to a unique word in the corpus. The value in each dimension represents the frequency of the word in the document. This approach is simple and efficient but ignores the semantic meaning and word order.

In [19]:
from sklearn.feature_extraction.text import CountVectorizer

# Initialize the CountVectorizer
vectorizer = CountVectorizer()

# Fit the vectorizer on the training data and transform the training data into a BoW representation
X_train_bow = vectorizer.fit_transform(X_train)

# Transform the test data into a BoW representation
X_test_bow = vectorizer.transform(X_test)


classifier = LogisticRegression()
classifier.fit(X_train_bow, y_train)

# Predict on the test data
y_pred = classifier.predict(X_test_bow)

# Evaluate the performance of the classifier
accuracy = classifier.score(X_test_bow, y_test)
print("Accuracy:", accuracy)

Accuracy: 0.8203165950309257


/nas.dbms/buyan/miniconda3/envs/bulltorch/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Term Frequency-Inverse Document Frequency (TF-IDF): TF-IDF is similar to BoW but takes into account the importance of words in a document and across the entire corpus. It assigns higher weights to words that are more frequent in a document but less frequent in the entire corpus. TF-IDF is a useful technique to identify important and distinctive words in a document.

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer
# Initialize the TfidfVectorizer
vectorizer = TfidfVectorizer()

# Fit the vectorizer on the training data and transform the training data into a TF-IDF representation
X_train_tfidf = vectorizer.fit_transform(X_train)

# Transform the test data into a TF-IDF representation
X_test_tfidf = vectorizer.transform(X_test)

# Initialize and train a classifier (e.g., Logistic Regression) using the TF-IDF representation
classifier = LogisticRegression()
classifier.fit(X_train_tfidf, y_train)

# Predict on the test data
y_pred = classifier.predict(X_test_tfidf)

# Evaluate the performance of the classifier
accuracy = classifier.score(X_test_tfidf, y_test)
print("Accuracy:", accuracy)

Accuracy: 0.8197924310724395


/nas.dbms/buyan/miniconda3/envs/bulltorch/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
